# 需要把程式簡化，這是新增的檔案一次寫入MySQL。

In [197]:
import csv,io,os,re
import sys
#載入sys是因為要重啟sys預設編碼改成'utf-8'，下面的default等等是因為reload sys的話print在jupyter會印不出來，再重新設定
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')


import mysql.connector

cnx = mysql.connector.connect(user='root',password='apple', database='insert')
cursor = cnx.cursor()

files=[]
for f in os.listdir('./clean/'):
    if os.path.isfile('./clean/'+f):
        files.append(f)
replace2=[]
for f in files:
    if '.csv' in f:
        #利用正規表達式，從檔名去找table名
        match=re.findall('[A-Z0-9]+',f)
        tablename=str(match[0])
#         print tablename
        with io.open('./clean/'+f,'r',encoding='utf-8') as rcsv:
            content=[]
            for line in rcsv:
                content.append(line)
            for element in content[1:]:
                #不需要第一行標題 [1:]
#                 print element.split(',')
#                 print mytuple
                mylist=[tablename,]#先用list再轉成tuple
                for ele in element.split(','):
#                     print str(ele)
                    mylist.append(str(ele).strip())
                mytuple = tuple(mylist)#轉成tuple
#                 print mytuple
                replace2.append(mytuple) 
# print replace2[0][2]
           
#這裡要注意的是'{r[1]}','{r[2]}'...等等到r5要加引號是因為在query時就要加''號，因為'00:00:00'沒加''會出錯！！！
for ele in replace2:         
    query = ('''insert into {r[0]} (EQIPnumber,location,direct,YMD,hour,laneNumber,addTotal,15Total,
truckflow,carflow,scooterflow,avgspeed,avgPercent,avgCarSpace)values
('{r[1]}','{r[2]}','{r[3]}','{r[4]}','{r[5]}',{r[6]},{r[7]},{r[8]},{r[9]},{r[10]},{r[11]},{r[12]},{r[13]},{r[14]})
;'''.format(r=ele))
#     print query
    cursor.execute(query)
    cnx.commit()
cursor.close()

cnx.close()
